# This code documents the data manipulation and processing steps that create DRIP data based off USGS Dam Removal Science Database and American Rivers Dam Removal Database (initial import of data documented in DRIP Data Into Database.ipynb).  This code is a work in progress and will likely change with time.

In [1]:
import requests
import geopandas as gpd
import json
import geojson
import pandas as pd
import numpy as np
import time
from bis2 import gc2
import re

In [2]:
damHeight_ft = float(6)
damBuiltYear = int
damRemovedYear = int

In [3]:
#Retrieves Data for analytical units, Renders GEOJSON as Python object
queryAr = 'https://gc2.datadistillery.org/api/v1/sql/bcb?q=select a._id as id, a.ar_id, a.year_built as arBuilt, a.year_removed as arRemoved, a.dam_height_ft, a.dam_name, a.river, b.featureuri from drip.ardamremoval a left join drip.dripsb b on a.ar_id = b.ar_id' 
rAr = requests.get(url=queryAr,verify=False)
queryDrd = 'https://gc2.datadistillery.org/api/v1/sql/bcb?q=select a._id as id, a.ar_id, a.damyearbuiltoriginalstructure, a.damyearbuiltremovedstructure, a.damyearremovalfinished, a.damheight_m, a.damname, a.damrivername, b.featureuri from drip.drddamremovals a left join drip.dripsb b on a.damaccessionnumber=b.damaccessionnumber'
rDrd = requests.get(url=queryDrd,verify=False)

C:\Users\dwieferich\AppData\Local\Continuum\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\Users\dwieferich\AppData\Local\Continuum\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [4]:
arJson = json.loads(rAr.text)
arFeatures = arJson['features']
arData = []
for feature in arFeatures:
    arData.append(feature['properties'])

arDataDf = pd.DataFrame(arData)

In [5]:
drdJson = json.loads(rDrd.text)
drdFeatures = drdJson['features']
drdData = []
for feature in drdFeatures:
    drdData.append(feature['properties'])
    
drdDataDf = pd.DataFrame(drdData)

In [6]:
arDataDf

,ar_id,arbuilt,arremoved,dam_height_ft,dam_name,featureuri,id,river
0,CT-001,,,17,John Dee's Dam,https://www.sciencebase.gov/catalog/item/566b3...,80fcb8f0-55f7-484e-ba2f-4d948c7410df,Mad River
1,PA-130,1875,2007,16,Kehly Run Dam No. 2,https://www.sciencebase.gov/catalog/item/55fb4...,216594ae-d722-49b4-be02-52c2863eed2d,Kehly Run
2,PA-202,Mid-19th Century,2010,3.5,Smucker Dam,https://www.sciencebase.gov/catalog/item/55fb4...,d6a04165-f655-444d-8cda-1045ff8eac9f,Groff Run
3,PA-226,1740,2011,4,Unnamed (Right mill race) Dam,https://www.sciencebase.gov/catalog/item/55fb4...,40fc75ca-0a1f-45a5-b750-36a9c862edec,Tributary to Yellow Breeches
4,VA-015,,2005,15,Rockland Dam,https://www.sciencebase.gov/catalog/item/55fb4...,61960a99-40b9-4555-81f0-41039a2abb74,Shenandoah River Middle Branch
5,IL-016,,2004,8,YWCA Dam,https://www.sciencebase.gov/catalog/item/55fb4...,9bfe6154-e118-4ead-ba89-648c822c3e67,Brewster Creek
6,CT-010,,1995,8,Simpson?s Pond Dam,https://www.sciencebase.gov/catalog/item/55fb4...,683b0691-6f66-41ff-b03d-3ca5cc995282,Wharton Brook
7,VT-005,,1995,46,Youngs Brook Dam,https://www.sciencebase.gov/catalog/item/566b3...,fff3613f-a1a2-4919-9d6f-727d06e2ade1,Youngs Brook
8,PA-179,,2009,,Axe Factory Dam,https://www.sciencebase.gov/catalog/item/55fb4...,3585fed1-a69a-408c-97eb-c5c0a5fd3a3e,Fishing Creek
9,CA-006,,,10,Salt Creek Dam,https://www.sciencebase.gov/catalog/item/55fb4...,dbe0d2a7-db7f-4313-9ba0-8de199d57c1a,Salt Creek


In [7]:
arInDrd = drdDataDf['ar_id'].dropna().tolist()
arInDrd = list(filter(None, arInDrd)) # removes empty str from list

In [8]:
#This cell cleans american rivers data.  For example string characters are removed from integar fields
arClean = []

for row in arDataDf.itertuples():
    identifier = row.id
    ar_id = row.ar_id
    
    #set dam height
    height = row.dam_height_ft 
    if height == 0:
        damHeight_ft = None
    elif not height:
        damHeight_ft = None
    else:
        if '~' in height:
            height = re.sub('~','', height)
        try:
            damHeight_ft = float(height)
        except:
            damHeight_ft = None
            
    #set year built
    built = row.arbuilt
    if built == 0:
        damBuiltYear = None
    elif not built:
        damBuiltYear = None
    else:
        if '~' in built:
            built = re.sub('~','', built)
        if 's' in built:
            built = re.sub('s','', built)
        try:
            damBuiltYear = int(built)
        except:
            damBuiltYear = None
    
    #set year removed
    removed = row.arremoved
    if removed == 0:
        damRemovedYear = None
    elif not removed:
        damRemovedYear = None
    else:
        if '~' in removed:
            removed = re.sub('~','', removed)
        if 's' in removed:
            removed = re.sub('s','', removed)
        try:
            damRemovedYear = int(removed)
        except:
            damRemovedYear = None
            
    #set dam name and stream name
    #Need to deal with this dam name : damSong of the Morning Ranch Dam (Golden Lotus Dam; Lansing Club Dam)'
    dam_name = row.dam_name
    dam_name = dam_name.replace("'",'')
    dam_name = dam_name.replace('&','and')
    #These alternate names for one dam in the dam name field are causing issues in insert so I am dropping them
    dam_name = dam_name.replace("(Golden Lotus Dam; Lansing Club Dam)",'')
    
    stream_name = row.river
    stream_name = stream_name.replace("'",'')
    
    inDrd = 0
    
    dam_uri = row.featureuri

    arClean.append({'identifier': identifier, 'ar_id': ar_id, 'damBuiltYear': damBuiltYear, 'damRemovedYear': damRemovedYear, 'damHeight_ft': damHeight_ft, 'dam_name':dam_name, 'stream_name':stream_name, 'inDrd':inDrd, 'dam_uri': dam_uri})      

arCleanDf = pd.DataFrame(arClean)

In [9]:
drdClean = []

for row in drdDataDf.itertuples():
    
    #identifier
    identifier = row.id
    ar_id = row.ar_id
    #Convert meters to feet
    height = row.damheight_m
    if (height == 0 or not height) and ar_id in arInDrd:  #Ensure no zero values
        record = arCleanDf.loc[arCleanDf['ar_id'] == ar_id]
        try:
            damHeight_ft = float(record['damHeight_ft'])
        except:
            damHeight_ft = None
    elif height == 0 or not height:  #Ensure no zero values
        damHeight_ft = None
    elif height:
        damHeight_ft = float(height) * 3.28084   #convert meters to feet
    else:
        damHeight_ft = height
    
    #Combine Year Built fields
    built = row.damyearbuiltremovedstructure
    builtorig = row.damyearbuiltoriginalstructure
    if (built == 0 or not built) and ar_id in arInDrd:
        if builtorig and builtorig != 0:
            try:
                damBuiltYear = int(builtorig)
            except:
                record = arCleanDf.loc[arCleanDf['ar_id'] == ar_id]
                try:
                    damBuiltYear = int(record['damBuiltYear'])
                except:
                    damBuiltYear = None
    elif (built == 0 or not built):
        damBuiltYear = None
    else:
        try:
            damBuiltYear = int(built)
        except:
            damBuiltYear = None
        
    #Year Removed 
    removed = row.damyearremovalfinished
    if (removed == 0 or not removed) and ar_id in arInDrd:
        record = arCleanDf.loc[arCleanDf['ar_id'] == ar_id]
        try:
            damRemovedYear = int(record['damRemovedYear'])
        except:
            damRemovedYear = None
    elif removed == 0 or not removed:
        damRemovedYear = None
    else:
        try:
            damRemovedYear = int(removed)
        except:
            damRemovedYear = None
    
    #Set dam_name, stream_name, and dam_uri
    #Might want to add complexity here, if dam doesn't have dam_name use ardf
    dam_name = row.damname
    dam_name = dam_name.replace("'",'')
    dam_name = dam_name.replace('&','and')
    
    stream_name = row.damrivername
    stream_name = stream_name.replace("'",'')
    
    dam_uri = row.featureuri
    inDrd = 1
    
    drdClean.append({'identifier': identifier, 'ar_id': ar_id, 'damBuiltYear': damBuiltYear, 'damRemovedYear': damRemovedYear, 'damHeight_ft': damHeight_ft, 'dam_name':dam_name, 'stream_name':stream_name, 'dam_uri':dam_uri, 'inDrd':inDrd})

drdCleanDf = pd.DataFrame(drdClean)

### The next several cells are in place to manually verify table sizes coming out of AR and DRD cleaning and the combination of the two tables

In [10]:
drdCleanDf.shape

(170, 9)

In [11]:
arCleanDf.shape

(1392, 9)

In [12]:
for ar in arInDrd:
    arCleanDf = arCleanDf[arCleanDf['ar_id'].isin([ar])== False]
   

In [13]:
 arCleanDf.shape

(1257, 9)

In [14]:
len(arInDrd)

135

In [15]:
#Make sure length of arInDrd plus ar data shape adds up to initial ar shape
1257+135

1392

In [16]:
arCleanDf

,ar_id,damBuiltYear,damHeight_ft,damRemovedYear,dam_name,dam_uri,identifier,inDrd,stream_name
0,CT-001,NaN,17.0,NaN,John Dees Dam,https://www.sciencebase.gov/catalog/item/566b3...,80fcb8f0-55f7-484e-ba2f-4d948c7410df,0,Mad River
1,PA-130,1875.0,16.0,2007.0,Kehly Run Dam No. 2,https://www.sciencebase.gov/catalog/item/55fb4...,216594ae-d722-49b4-be02-52c2863eed2d,0,Kehly Run
2,PA-202,NaN,3.5,2010.0,Smucker Dam,https://www.sciencebase.gov/catalog/item/55fb4...,d6a04165-f655-444d-8cda-1045ff8eac9f,0,Groff Run
3,PA-226,1740.0,4.0,2011.0,Unnamed (Right mill race) Dam,https://www.sciencebase.gov/catalog/item/55fb4...,40fc75ca-0a1f-45a5-b750-36a9c862edec,0,Tributary to Yellow Breeches
4,VA-015,NaN,15.0,2005.0,Rockland Dam,https://www.sciencebase.gov/catalog/item/55fb4...,61960a99-40b9-4555-81f0-41039a2abb74,0,Shenandoah River Middle Branch
6,CT-010,NaN,8.0,1995.0,Simpson?s Pond Dam,https://www.sciencebase.gov/catalog/item/55fb4...,683b0691-6f66-41ff-b03d-3ca5cc995282,0,Wharton Brook
7,VT-005,NaN,46.0,1995.0,Youngs Brook Dam,https://www.sciencebase.gov/catalog/item/566b3...,fff3613f-a1a2-4919-9d6f-727d06e2ade1,0,Youngs Brook
8,PA-179,NaN,NaN,2009.0,Axe Factory Dam,https://www.sciencebase.gov/catalog/item/55fb4...,3585fed1-a69a-408c-97eb-c5c0a5fd3a3e,0,Fishing Creek
9,CA-006,NaN,10.0,NaN,Salt Creek Dam,https://www.sciencebase.gov/catalog/item/55fb4...,dbe0d2a7-db7f-4313-9ba0-8de199d57c1a,0,Salt Creek
10,NJ-011,NaN,NaN,2009.0,Seber Dam,https://www.sciencebase.gov/catalog/item/55fb4...,03dbfaf1-6def-4b0c-b90b-f3e770124009,0,Musconetcong River


In [17]:
#Append AR dams(not including those in USGS database) to USGS database
dripDf = drdCleanDf.append(arCleanDf)

In [18]:
dripDf.shape

(1427, 9)

In [19]:
dripDf

,ar_id,damBuiltYear,damHeight_ft,damRemovedYear,dam_name,dam_uri,identifier,inDrd,stream_name
0,WI-077,1952.0,8.858268,1996.0,Afton Dam,https://www.sciencebase.gov/catalog/item/55fb4...,9cddf5d4-89b7-4d11-95d2-ca75bffa96c9,1,Bass Creek
1,CT-011,1750.0,11.154856,1999.0,Anaconda Dam,https://www.sciencebase.gov/catalog/item/55fb4...,1b72adcb-3d13-43b4-89a3-e87b46cf0517,1,Naugatuck River
2,MN-007,1872.0,17.060368,1999.0,Appleton Dam,https://www.sciencebase.gov/catalog/item/55fb4...,57093f00-3f8a-4bf4-8e6c-0dc893986a9b,1,Pomme de Terre River
3,WI-115,1900.0,11.154856,2008.0,Big Spring Dam,https://www.sciencebase.gov/catalog/item/55fb4...,b3224aae-5b74-4b71-aa73-cef90cc505dd,1,Big Spring Creek
4,PA-309,1900.0,NaN,1916.0,Big Spring Run 1,https://www.sciencebase.gov/catalog/item/55fb4...,504c142f-36f8-4f8c-98ce-52a099e3d4a9,1,Big Spring Run
5,WI-006,1851.0,14.000000,2012.0,Black Earth Dam,https://www.sciencebase.gov/catalog/item/55fb4...,bc452c50-30b3-40c1-88c0-86ecc4ba19f3,1,Black Earth Creek
6,WI-096,1955.0,8.202100,2003.0,"Boulder Creek ""lower dam""",https://www.sciencebase.gov/catalog/item/55fb4...,bd3bdc3f-b724-4f2a-8f91-5e8c4d825d28,1,Boulder Creek
7,WI-095,1955.0,3.280840,2003.0,"Boulder Creek ""upper dam""",https://www.sciencebase.gov/catalog/item/55fb4...,ae786fdb-c068-4f52-ac9a-cbdf71385e30,1,Boulder Creek
8,WI-054,1851.0,12.000000,1996.0,Bowen Mills Dam,https://www.sciencebase.gov/catalog/item/55fb4...,9dcd13c6-a1dc-4042-be62-e30223e0cada,1,Pine River
9,OR-017,1965.0,8.202100,2007.0,Brownsville Dam,https://www.sciencebase.gov/catalog/item/55fb4...,0b771582-010e-42d9-bb10-da0e612a8e18,1,Calapooia River


In [20]:
#Build a new table in GC2 if it doesn't already exist
q_createDripDams = "CREATE TABLE IF NOT EXISTS drip.dripdams( \
    _id varchar(40) primary key,\
    damBuiltYear int, \
    damRemovedYear int, \
    damHeight_ft float(6), \
    dam_name varchar(150), \
    stream_name varchar(150), \
    dam_uri varchar(80), \
    inDrd int)"
url_createDripDams = gc2.sqlAPI("datadistillery","bcb")+"&q="+q_createDripDams
print (requests.get(url_createDripDams,verify=False).json())

{'success': True, 'affected_rows': 0, '_execution_time': 0.024}


C:\Users\dwieferich\AppData\Local\Continuum\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [22]:
#Upload final dripdams table into GC2

# Set up the actions/targets for this particular instance
thisRun = {}
thisRun['instance'] = 'DataDistillery'
thisRun['db'] = 'BCB'
thisRun['baseURL'] = gc2.sqlAPI(thisRun['instance'],thisRun['db'])  #base url includes ?key
thisRun['schema'] = 'drip'
thisRun['commitToDB'] = False
thisRun['Table'] = 'dripdams'
thisRun['gc2Key'] = gc2.gc2Keys['datadistillery_bcb']  #Reads in api key, allowing for insert capabilities

q_recordToSearch = "SELECT _id as id from drip.dripdams1" 
recordToSearch = requests.get(gc2.baseURLs["sqlapi_datadistillery_bcb"]+"?q="+q_recordToSearch, verify=False).json()
p = recordToSearch['features']
lookup = []
for f in p:
    s = str(f['properties']['id'])
    lookup.append(s)

lenList = len(lookup)
cntExisting = 0 
existingList = []

for row in dripDf.itertuples():
    if row.identifier in lookup:
        continue
    else:
        identifier = str(row.identifier)
        #print (str(row.damRemovedYear))
        removed = str(row.damRemovedYear)
        if removed == 'nan':
            removed = 'null'
        height = str(row.damHeight_ft)   
        if height == 'nan':
            height = 'null'
        built = str(row.damBuiltYear)
        if built == 'nan':
            built = 'null'
        dam_name = str(row.dam_name)
        stream_name = str(row.stream_name)
        if stream_name == 'None' or stream_name == 'nan':
            stream_name = ' '
        dam_uri = str(row.dam_uri)
        if dam_uri == 'None' or dam_uri == 'nan':
            dam_uri = ' '
        inDrd = str(row.inDrd)
        
        q = "insert into drip.dripdams1(_id, damBuiltYear, damRemovedYear, damHeight_ft, \
        dam_name, stream_name, dam_uri, inDrd)\
        VALUES ('" + identifier + "' ,"  + built + " ," + removed + " ," + height + " ,'" + dam_name \
        + "' ,'" + stream_name + "' ,'" + dam_uri + "'," + inDrd + ")"
        payload = "q=%s&key=%s"%(q,thisRun['gc2Key'])
        #print (payload)
        url= gc2.baseURLs["sqlapi_datadistillery_bcb"]
        #print (url)
        
        try:
            r = requests.post(url,data=payload,verify=False)
            print (r)
        except:
            continue
        

C:\Users\dwieferich\AppData\Local\Continuum\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


#Below here documents beta specific cells

In [ ]:
#Retrieves Data for analytical units, Renders GEOJSON as Python object
queryAr = 'https://beta-gc2.datadistillery.org/api/v1/sql/bcb?q=select a._id as id, a.ar_id, a.year_built as arBuilt, a.year_removed as arRemoved, a.dam_height_ft, a.dam_name, a.river, b.featureuri from drip.ardamremoval a left join drip.dripSb b on a.ar_id = b.ar_id' 
rAr = requests.get(url=queryAr,verify=False)
queryDrd = 'https://beta-gc2.datadistillery.org/api/v1/sql/bcb?q=select a._id as id, a.ar_id, a.damyearbuiltoriginalstructure, a.damyearbuiltremovedstructure, a.damyearremovalfinished, a.damheight_m, a.damname, a.damrivername, b.featureuri from drip.drddamremovals a left join drip.dripsb b on a.damaccessionnumber=b.damaccessionnumber'
rDrd = requests.get(url=queryDrd,verify=False)

In [ ]:
#Build a new table in GC2 if it doesn't already exist
q_createDripDams = "CREATE TABLE IF NOT EXISTS drip.dripdams( \
    _id varchar(40) primary key,\
    damBuiltYear int, \
    damRemovedYear int, \
    damHeight_ft float(6), \
    dam_name varchar(150), \
    stream_name varchar(150), \
    dam_uri varchar(80), \
    inDrd int)"
url_createDripDams = gc2.sqlAPI("datadistillery","bcb_beta")+"&q="+q_createDripDams
print (requests.get(url_createDripDams,verify=False).json())

In [ ]:
# Set up the actions/targets for this particular instance


#BETA BETA BETA BETA BETA




thisRun = {}
thisRun['instance'] = 'DataDistillery'
thisRun['db'] = 'BCB_beta'
thisRun['baseURL'] = gc2.sqlAPI(thisRun['instance'],thisRun['db'])  #base url includes ?key
thisRun['schema'] = 'drip'
thisRun['commitToDB'] = False
thisRun['Table'] = 'drip_height_years'
thisRun['gc2Key'] = gc2.gc2Keys['datadistillery_bcb_beta']  #Reads in api key, allowing for insert capabilities

q_recordToSearch = "SELECT _id as id from drip.dripdams" 
recordToSearch = requests.get(gc2.baseURLs["sqlapi_datadistillery_bcb_beta"]+"?q="+q_recordToSearch, verify=False).json()
p = recordToSearch['features']
lookup = []
for f in p:
    s = str(f['properties']['id'])
    lookup.append(s)
lenList = len(lookup)
cntExisting = 0 
existingList = []

for row in dripDf.itertuples():
    if row.identifier in lookup:
        continue
    else:
        identifier = str(row.identifier)
        #print (str(row.damRemovedYear))
        removed = str(row.damRemovedYear)
        if removed == 'nan':
            removed = 'null'
        height = str(row.damHeight_ft)   
        if height == 'nan':
            height = 'null'
        built = str(row.damBuiltYear)
        if built == 'nan':
            built = 'null'
        dam_name = str(row.dam_name)
        stream_name = str(row.stream_name)
        if stream_name == 'None' or stream_name == 'nan':
            stream_name = ' '
        dam_uri = str(row.dam_uri)
        if dam_uri == 'None' or dam_uri == 'nan':
            dam_uri = ' '
        inDrd = str(row.inDrd)
        
        q = "insert into drip.dripdams(_id, damBuiltYear, damRemovedYear, damHeight_ft, \
        dam_name, stream_name, dam_uri, inDrd)\
        VALUES ('" + identifier + "' ,"  + built + " ," + removed + " ," + height + " ,'" + dam_name \
        + "' ,'" + stream_name + "' ,'" + dam_uri + "'," + inDrd + ")"
        payload = "q=%s&key=%s"%(q,thisRun['gc2Key'])
        print (payload)
        url= gc2.baseURLs["sqlapi_datadistillery_bcb_beta"]
        #print (url)
        
        try:
            r = requests.post(url,data=payload,verify=False)
            print (r)
        except:
            continue
        